# Import necessary libraries

In [1]:
#import necessary libraries
import pandas as pd
import numpy as np
import os
import random as rand

# Custom functions that we will use during this project

In [2]:
#create a function to find outliers using IQR
def find_outliers_IQR(df):
    #this function takes a dataframe or only a column and calculates outliers for each value based on IQR
    q1=df.quantile(0.25) # variable for the first quartile
    q3=df.quantile(0.75) #variable for the third quartile
    IQR=q3-q1 #calculate the IQR
    outliers = df[((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))] #calculate outliers for every value and put them in a series
    return outliers

#Range function
def range_column(df):
    max = df.max()
    min = df.min()
    range = max-min
    return range

# Import the raw data via pandas

In [3]:
#read the excel file and create a data frame
dataFrame = pd.read_excel('movie_data.xlsx')

# Display descriptive and technical info about the raw, dirty data

In [4]:
#display the head
dataFrame.head()

,genre,gross,cost,no_of_viewer,year,duration,outside,netflix,rating
0,Action,2.556502e+06,7.432357e+06,NaN,2010.0,292.221364,no,yes,3.111750
1,NaN,1.991826e+07,2.148195e+07,1.330288e+07,2014.0,290.725164,yes,no,6.194811
2,Adventure,1.456786e+07,1.747146e+07,1.136820e+07,2021.0,259.883566,no,YeS,5.265078
3,Drama,1.855628e+07,1.991671e+07,1.272027e+07,2016.0,275.505929,yes,NaN,5.921890
4,Adventure,1.393263e+07,NaN,1.105232e+07,NaN,263.948467,NaN,yes,5.119858


In [5]:
#the tail
dataFrame.tail()

,genre,gross,cost,no_of_viewer,year,duration,outside,netflix,rating
195,0,1.633701e+07,1.852572e+07,1.196560e+07,2022.0,269.988267,NaN,NaN,5.554348
196,0,8.441605e+06,1.297282e+07,9.160656e+06,2007.0,267.939379,no,yes,4.208167
197,0,9.599373e+06,NaN,9.608058e+06,2003.0,260.171831,no,NaN,NaN
198,0,1.240572e+07,1.539260e+07,NaN,2012.0,NaN,no,yes,4.860113
199,0,2.284349e+06,8.161214e+06,6.899894e+06,2005.0,285.537877,no,NaN,3.125985


In [6]:
#NA values
dataFrame.isna().sum()

genre           36
gross           31
cost            31
no_of_viewer    27
year            33
duration        23
outside         33
netflix         27
rating          28
dtype: int64

In [7]:
#display info about the dataframe
dataFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genre         164 non-null    object 
 1   gross         169 non-null    float64
 2   cost          169 non-null    float64
 3   no_of_viewer  173 non-null    float64
 4   year          167 non-null    float64
 5   duration      177 non-null    float64
 6   outside       167 non-null    object 
 7   netflix       173 non-null    object 
 8   rating        172 non-null    float64
dtypes: float64(6), object(3)
memory usage: 14.2+ KB


We have 6 float value columns and 3 object value columns, which seems appropriate with the datatypes for the given columns. We decided to keep the year as float as we only have the year and not a full date.  Data seems to be imported fine. Column headers are values. We don't have multiple variables in a column. Both rows and columns hold values. Every column has the same unit in itself. 

Titles look broken and weird. Let's fix that.

# Make titles nicer

In [8]:
#take the column names and turn them into capitalized titles
dataFrame.columns = dataFrame.columns.str.title()
dataFrame.columns #display titles to see changes

Index(['Genre', 'Gross', 'Cost', 'No_Of_Viewer', 'Year', 'Duration', 'Outside',
       'Netflix', 'Rating'],
      dtype='object')

In [9]:
#rename 'No_Of_Viewer' column as 'Number_of_Viewers'
dataFrame = dataFrame.rename({'No_Of_Viewer' : 'Number of Viewers'},axis='columns')
dataFrame.columns

Index(['Genre', 'Gross', 'Cost', 'Number of Viewers', 'Year', 'Duration',
       'Outside', 'Netflix', 'Rating'],
      dtype='object')

# check for duplicates and unique values (will check again later)

In [10]:
#examine duplicates
dataFrame.duplicated().sum()

0

It seems like that we have no duplicates, but we will make sure that we won't have any in the future steps

In [11]:
#check for the number of unique values
dataFrame.nunique()

Genre                  7
Gross                169
Cost                 169
Number of Viewers    173
Year                  23
Duration             177
Outside                5
Netflix                5
Rating               163
dtype: int64

It looks like we have some non-unique values.We will check this later when we fill NAs and replace outliers (spoilers!)

## object type columns

In [12]:
#count the values for columns with object datatype
for i in dataFrame.select_dtypes(include = 'object').columns:
    print(dataFrame[i].value_counts()) 

0            83
Adventure    35
Action       18
Drama        15
ADVenture     8
 DRAMa        3
ACtion        2
Name: Genre, dtype: int64
no     116
yes     38
NO       7
 NO      5
YeS      1
Name: Outside, dtype: int64
yes     119
no       41
YeS       7
 YeS      5
NO        1
Name: Netflix, dtype: int64


The values look like an alphabet soup! We should unify them.

Also, genre column looks weird... 83 of the values are 0.

### Genre column

In [13]:
#determine the amount of NA cells in Genre column
genre_na = dataFrame['Genre'].isna().sum()
print("NA values: ", genre_na)

#find the amount of 0s
genre_zero = dataFrame['Genre'].value_counts()[0]
print("0 values: ", genre_zero)

#see the total number of garbage values
genre_sum = genre_na + genre_zero
print("Total garbage: ", genre_sum)

#also find the % proportion of garbages
proportion = (genre_sum * 100) // len(dataFrame.index)
print("Proportions: ", proportion,"%")

NA values:  36
0 values:  83
Total garbage:  119
Proportions:  59 %


Checklist says drop the column if we have 60-65% of the data unusable. 59% is pretty much the same as 60% as our sample size isn't gigantic.

In [14]:
#drop the Genre column as more than half of the column is filled with 0 and NA
dataFrame = dataFrame.drop(columns = ['Genre'])

#let's see how it looks like now
dataFrame.head()

,Gross,Cost,Number of Viewers,Year,Duration,Outside,Netflix,Rating
0,2.556502e+06,7.432357e+06,NaN,2010.0,292.221364,no,yes,3.111750
1,1.991826e+07,2.148195e+07,1.330288e+07,2014.0,290.725164,yes,no,6.194811
2,1.456786e+07,1.747146e+07,1.136820e+07,2021.0,259.883566,no,YeS,5.265078
3,1.855628e+07,1.991671e+07,1.272027e+07,2016.0,275.505929,yes,NaN,5.921890
4,1.393263e+07,NaN,1.105232e+07,NaN,263.948467,NaN,yes,5.119858


### Outside column

In [15]:
#determine the amount of NA cells in Outside column
outside_na = dataFrame['Outside'].isna().sum()
print("NA values: ", outside_na)

#also find the % proportion of garbages
proportion = (outside_na * 100) // len(dataFrame.index)
print("Proportions: ", proportion,"%")

NA values:  33
Proportions:  16 %


Seems fine.

In [16]:
#correct the misspelled yes and no values in Outside column and display NA cells

#strip the whitespaces
dataFrame['Outside'] = dataFrame['Outside'].str.strip()

#correct yes's and no's
dataFrame['Outside'] = dataFrame['Outside'].replace(to_replace = ['NO', 'no'], value = 'No')
dataFrame['Outside'] = dataFrame['Outside'].replace(to_replace = ['YeS', 'yes'], value = 'Yes')

#print NA and other values to see if it worked
print('NA values: ', dataFrame['Outside'].isna().sum())
print(dataFrame['Outside'].value_counts())

NA values:  33
No     128
Yes     39
Name: Outside, dtype: int64


Let's fill the NAs of Outside. We can use binomial distribution to fill the NAs.

In [17]:
# let's find the probability of Yes
# first find the amount of filled cells

filled = len(dataFrame.index) - dataFrame['Outside'].isna().sum() # the cells that are filled already

# probability of choosing yes, take as p
p = (dataFrame['Outside'].value_counts()['Yes']) / filled
print("Prob. Yes: ", p)

# fill NA values randomly based on the probability of yes
for i in range(dataFrame['Outside'].isna().sum()):
    random = rand.random()
    if random <= p:
        dataFrame['Outside'] = dataFrame['Outside'].fillna('Yes')
    else:
        dataFrame['Outside'] = dataFrame['Outside'].fillna('No')

#Let's see NA values again
print('NA values: ', dataFrame['Outside'].isna().sum())

Prob. Yes:  0.23353293413173654
NA values:  0


### Netflix column

In [18]:
#determine the amount and proportion of NA cells in Netflix column
netflix_na = dataFrame['Netflix'].isna().sum()
print("NA values: ", netflix_na)

#find the % proportion of garbages
proportion = (netflix_na * 100) // len(dataFrame.index)
print("Proportions: ", proportion,"%")

NA values:  27
Proportions:  13 %


Looks OK.

In [19]:
#correct the misspelled yes and no values in Netflix column and display NA cells

#strip the whitespaces
dataFrame['Netflix'] = dataFrame['Netflix'].str.strip()

#correct yes's and no's
dataFrame['Netflix'] = dataFrame['Netflix'].replace(to_replace = ['NO', 'no'], value = 'No')
dataFrame['Netflix'] = dataFrame['Netflix'].replace(to_replace = ['YeS', 'yes'], value = 'Yes')

#print NA and other values to see if it worked
print('NA values: ', dataFrame['Netflix'].isna().sum())
print(dataFrame['Netflix'].value_counts())

NA values:  27
Yes    131
No      42
Name: Netflix, dtype: int64


We have NA values to fill in Netflix. Let's fill them now the same way we filled Outside.

In [20]:
# let's find the probability of Yes
# first find the amount of filled cells

filled = len(dataFrame.index) - dataFrame['Netflix'].isna().sum() # the cells that are filled already

# probability of choosing yes, take as p
p = (dataFrame['Netflix'].value_counts()['Yes']) / filled
print("Prob. Yes: ", p)

# fill NA values randomly based on the probability of yes
for i in range(dataFrame['Netflix'].isna().sum()):
    random = rand.random()
    if random <= p:
        dataFrame['Netflix'] = dataFrame['Netflix'].fillna('Yes')
    else:
        dataFrame['Netflix'] = dataFrame['Netflix'].fillna('No')

#Let's see NA values again
print('NA values: ', dataFrame['Netflix'].isna().sum())

Prob. Yes:  0.7572254335260116
NA values:  0


## float type columns

In [21]:
#Display number of negative values in each column
print('Negative(Gross)\t\t\t',(dataFrame['Gross'] < 0).sum().sum())
print('Negative(Cost)\t\t\t',(dataFrame['Cost'] < 0).sum().sum())
print('Negative(Num. of Viewers)\t',(dataFrame['Number of Viewers'] < 0).sum().sum())
print('Negative(Duration)\t\t',(dataFrame['Duration'] < 0).sum().sum())
print('Negative(Rating)\t\t',(dataFrame['Rating'] < 0).sum().sum())
print('\n----------------------------------------------------')

Negative(Gross)			 18
Negative(Cost)			 1
Negative(Num. of Viewers)	 0
Negative(Duration)		 0
Negative(Rating)		 0

----------------------------------------------------


In [22]:
#Take absolute value
dataFrameAbs = dataFrame.select_dtypes(include = 'float64').abs()
dataFrame['Gross'] = dataFrameAbs['Gross']
dataFrame['Cost'] = dataFrameAbs['Cost']
dataFrame['Number of Viewers'] = dataFrameAbs['Number of Viewers']
dataFrame['Duration'] = dataFrameAbs['Duration']
dataFrame['Rating'] = dataFrameAbs['Rating']

In [23]:
#Display number of negative values in each columns again to see whether absolute value command worked or not
print('Negative(Gross)\t\t\t',(dataFrame['Gross'] < 0).sum().sum())
print('Negative(Cost)\t\t\t',(dataFrame['Cost'] < 0).sum().sum())
print('Negative(Num. of Viewers)\t',(dataFrame['Number of Viewers'] < 0).sum().sum())
print('Negative(Duration)\t\t',(dataFrame['Duration'] < 0).sum().sum())
print('Negative(Rating)\t\t',(dataFrame['Rating'] < 0).sum().sum())
dataFrame.isna().sum()

Negative(Gross)			 0
Negative(Cost)			 0
Negative(Num. of Viewers)	 0
Negative(Duration)		 0
Negative(Rating)		 0


Gross                31
Cost                 31
Number of Viewers    27
Year                 33
Duration             23
Outside               0
Netflix               0
Rating               28
dtype: int64

### Gross column

In [24]:
#determine the amount and proportion of NA cells in Gross column
gross_na = dataFrame['Gross'].isna().sum()
print("NA values: ", gross_na)

#find the % proportion of garbages
proportion = (gross_na * 100) // len(dataFrame.index)
print("Proportions: ", proportion,"%")

NA values:  31
Proportions:  15 %


Looks OK.

In [25]:
#display NA values
print('Num. of NA (Gross): ', dataFrame['Gross'].isna().sum())

#Fill NA values in Gross with Median
median_of_gross = dataFrame['Gross'].median()
print('Median of gross: ', median_of_gross)
dataFrame['Gross'] = dataFrame['Gross'].fillna(median_of_gross)
print('Num. of NA (Gross): ', dataFrame['Gross'].isna().sum())

Num. of NA (Gross):  31
Median of gross:  10121202.5381579
Num. of NA (Gross):  0


In [26]:
#Display outliers of Gross column and replace them with the median
out_gross = find_outliers_IQR(dataFrame['Gross'])
print('Outliers (Gross)\n', str(out_gross))

for i in out_gross.index:
    val = out_gross[i]
    dataFrame['Gross'] = dataFrame['Gross'].replace(to_replace = val, value = median_of_gross)
    print("Replaced Outliers\n")
    print(i, "\t", dataFrame['Gross'][i])

Outliers (Gross)
 49     2.736197e+07
114    2.678449e+07
Name: Gross, dtype: float64
Replaced Outliers

49 	 10121202.5381579
Replaced Outliers

114 	 10121202.5381579


In [27]:
#Gross info
dataFrame['Gross'].describe().transpose()

count    2.000000e+02
mean     1.047595e+07
std      6.021160e+06
min      4.791287e+04
25%      6.493311e+06
50%      1.012120e+07
75%      1.398960e+07
max      2.537938e+07
Name: Gross, dtype: float64

### Cost column

In [28]:
#determine the amount and proportion of NA cells in Cost column
cost_na = dataFrame['Cost'].isna().sum()
print("NA values: ", cost_na)

#find the % proportion of garbages
proportion = (cost_na * 100) // len(dataFrame.index)
print("Proportions: ", proportion,"%")

NA values:  31
Proportions:  15 %


Seems good.

In [29]:
#display NA values
print('Num. of NA (Cost):\t', dataFrame['Cost'].isna().sum())

#Fill NA values in Cost with Median
median_of_cost = dataFrame['Cost'].median()
print('Median of Cost: ', median_of_cost)
dataFrame['Cost'] = dataFrame['Cost'].fillna(median_of_cost)
print('Num. of NA (Cost):\t', dataFrame['Cost'].isna().sum())

Num. of NA (Cost):	 31
Median of Cost:  13616744.7344021
Num. of NA (Cost):	 0


In [30]:
#Display outliers of Cost column and replace them with the median
out_cost = find_outliers_IQR(dataFrame['Cost'])
print('Outlier (Cost)\n', str(out_cost))

for i in out_cost.index:
    val = out_cost[i]
    dataFrame['Cost'] = dataFrame['Cost'].replace(to_replace = val, value = median_of_cost)
    print("Replaced Outliers\n")
    print(i, "\t", dataFrame['Cost'][i])

Outlier (Cost)
 49     2.641508e+07
58     7.201403e+05
114    2.585029e+07
165    4.151219e+03
185    9.354493e+05
Name: Cost, dtype: float64
Replaced Outliers

49 	 13616744.7344021
Replaced Outliers

58 	 13616744.7344021
Replaced Outliers

114 	 13616744.7344021
Replaced Outliers

165 	 13616744.7344021
Replaced Outliers

185 	 13616744.7344021


In [31]:
#Cost info
dataFrame['Cost'].describe().transpose()

count    2.000000e+02
mean     1.386937e+07
std      4.736942e+06
min      2.754498e+06
25%      1.125519e+07
50%      1.361674e+07
75%      1.685935e+07
max      2.495247e+07
Name: Cost, dtype: float64

### Number of Viewers column

In [32]:
#determine the amount and proportion of NA cells in Number of Viewers column
nov_na = dataFrame['Number of Viewers'].isna().sum()
print("NA values: ", nov_na)

#find the % proportion of garbages
proportion = (nov_na * 100) // len(dataFrame.index)
print("Proportions: ", proportion,"%")

NA values:  27
Proportions:  13 %


Looks smooth.

In [33]:
#display NA values
print('Num. of NA (Number of Viewers): ', dataFrame['Number of Viewers'].isna().sum())

#Fill NA values with Median
median_of_nov = dataFrame['Number of Viewers'].median()
print('Median of Number of Viewers: ', median_of_nov)
dataFrame['Number of Viewers'] = dataFrame['Number of Viewers'].fillna(median_of_nov)
print('Num. of NA (Number of Viewers): ', dataFrame['Number of Viewers'].isna().sum())

Num. of NA (Number of Viewers):  27
Median of Number of Viewers:  9576807.56792578
Num. of NA (Number of Viewers):  0


In [34]:
#Display outliers of Number of Viewers column and replace them with the median
out_nov = find_outliers_IQR(dataFrame['Number of Viewers'])
print('Outlier(Number of Viewers)\n', str(out_nov))

for i in out_nov.index:
    val = out_nov[i]
    dataFrame['Number of Viewers'] = dataFrame['Number of Viewers'].replace(to_replace = val, value = median_of_nov)
    print("Replaced Outliers\n")
    print(i, "\t", dataFrame['Number of Viewers'][i])

Outlier(Number of Viewers)
 18     2.418048e+06
49     1.588776e+07
129    3.557401e+06
165    2.919316e+06
185    3.561613e+06
Name: Number of Viewers, dtype: float64
Replaced Outliers

18 	 9576807.56792578
Replaced Outliers

49 	 9576807.56792578
Replaced Outliers

129 	 9576807.56792578
Replaced Outliers

165 	 9576807.56792578
Replaced Outliers

185 	 9576807.56792578


In [35]:
#Number of Viewers info
dataFrame['Number of Viewers'].describe().transpose()

count    2.000000e+02
mean     9.764626e+06
std      2.360164e+06
min      4.262467e+06
25%      8.444537e+06
50%      9.576808e+06
75%      1.124182e+07
max      1.567518e+07
Name: Number of Viewers, dtype: float64

### Year column

In [36]:
#determine the amount and proportion of NA cells in Year column
year_na = dataFrame['Year'].isna().sum()
print("NA values: ", year_na)

#find the % proportion of garbages
proportion = (year_na * 100) // len(dataFrame.index)
print("Proportions: ", proportion,"%")

NA values:  33
Proportions:  16 %


Looks nice.

In [37]:
# detect median of Year
median_of_year = dataFrame['Year'].median()
print('Median of Year: ', median_of_year)

Median of Year:  2011.0


In [38]:
yearModes = dataFrame['Year'].mode()
yearModes
# year is bi-modal

0    2003.0
1    2009.0
dtype: float64

In [39]:
#fill half of the NAs in Year with either modes, based on our instructors advice

na = dataFrame['Year'].isna().sum()
dataFrame['Year'] = dataFrame['Year'].fillna(value = yearModes[0], limit = na//2) #fill first half with the first mode
dataFrame['Year'].isna().sum()
    
#See NA values
print("NAs after first fill: ", dataFrame['Year'].isna().sum())
    
dataFrame['Year'] = dataFrame['Year'].fillna(value=yearModes[1]) #fill remaining NAs with the second mode
dataFrame['Year'].isna().sum()
    
#See NA values
print("NAs after second fill: ", dataFrame['Year'].isna().sum())

NAs after first fill:  17
NAs after second fill:  0


In [40]:
#Display outliers of Year column and replace them with the median
out_year = find_outliers_IQR(dataFrame['Year'])
print('Outlier(Year)\n', str(out_year))

for i in out_year.index:
    val = out_year[i]
    dataFrame['Year'] = dataFrame['Year'].replace(val, median_of_year)
    print("Replaced Outliers\n")
    print(i, "\t", dataFrame['Year'][i])

Outlier(Year)
 Series([], Name: Year, dtype: float64)


Good news! Looks like we have no outliers in Year.

In [41]:
#Year info
dataFrame['Year'].describe().transpose()

count     200.0000
mean     2010.1000
std         6.2751
min      2000.0000
25%      2003.7500
50%      2009.0000
75%      2015.0000
max      2022.0000
Name: Year, dtype: float64

### Duration column

In [42]:
#determine the amount and proportion of NA cells in Duration column
dur_na = dataFrame['Duration'].isna().sum()
print("NA values: ", dur_na)

#find the % proportion of garbages
proportion = (dur_na * 100) // len(dataFrame.index)
print("Proportions: ", proportion,"%")

NA values:  23
Proportions:  11 %


Seems fine.

In [43]:
#display NA values
print('Num. of NA (Duration): ', dataFrame['Duration'].isna().sum())

#Fill NA values with Median
median_of_dur = dataFrame['Duration'].median()
print('Median of Duration: ', median_of_dur)
dataFrame['Duration'] = dataFrame['Duration'].fillna(median_of_dur)
print('Num. of NA (Duration): ', dataFrame['Duration'].isna().sum())

Num. of NA (Duration):  23
Median of Duration:  273.190074859357
Num. of NA (Duration):  0


In [44]:
#Display outliers of Duration column and replace them with the median
out_dur = find_outliers_IQR(dataFrame['Duration'])
print('Outlier(Duration)\n', str(out_dur))

for i in out_dur.index:
    val = out_dur[i]
    dataFrame['Duration'] = dataFrame['Duration'].replace(to_replace = val, value = median_of_dur)
    print("Replaced Outliers\n")
    print(i, "\t", dataFrame['Duration'][i])

Outlier(Duration)
 64    239.522141
Name: Duration, dtype: float64
Replaced Outliers

64 	 273.190074859357


In [45]:
#Duration info
dataFrame['Duration'].describe().transpose()

count    200.000000
mean     274.370429
std       12.645504
min      243.685578
25%      266.820754
50%      273.190075
75%      282.852908
max      306.783128
Name: Duration, dtype: float64

### Rating column

In [46]:
# get the rating column in the range that is written in our data description, between 1 and 10
df_inrange = dataFrame[(dataFrame['Rating'] <= 10) & (dataFrame['Rating'] >= 1)]
print('Num. of NA (Rating in range): ', df_inrange['Rating'].isna().sum())
dataFrame['Rating'] = df_inrange['Rating']

Num. of NA (Rating in range):  0


In [47]:
#determine the amount and proportion of NA cells in Year column
rat_na = dataFrame['Rating'].isna().sum()
print("NA values: ", rat_na)

#find the % proportion of garbages
proportion = (rat_na * 100) // len(dataFrame.index)
print("Proportions: ", proportion,"%")

NA values:  39
Proportions:  19 %


In [48]:
#display NA values
print('Num. of NA (Rating): ', dataFrame['Rating'].isna().sum())

#Fill NA values with Median
median_of_rat = dataFrame['Rating'].median()
print('Median of Rating: ', median_of_rat)
dataFrame['Rating'] = dataFrame['Rating'].fillna(median_of_rat)
print('Num. of Na(Rating): ', dataFrame['Rating'].isna().sum())

Num. of NA (Rating):  39
Median of Rating:  4.41009717114297
Num. of Na(Rating):  0


In [49]:
#Display outliers of Rating column and replace them with the median
out_rat = find_outliers_IQR(dataFrame['Rating'])
print('Outlier(Rating)\n', str(out_rat))

for i in out_rat.index:
    val = out_rat[i]
    dataFrame['Rating'] = dataFrame['Rating'].replace(to_replace = val, value = median_of_rat)
    print("Replaced Outliers\n")
    print(i, "\t", dataFrame['Rating'][i])

Outlier(Rating)
 49     7.445626
58     1.609460
71     1.863155
104    1.659008
114    7.336382
125    7.091118
129    1.529494
165    1.214859
185    1.531479
Name: Rating, dtype: float64
Replaced Outliers

49 	 4.41009717114297
Replaced Outliers

58 	 4.41009717114297
Replaced Outliers

71 	 4.41009717114297
Replaced Outliers

104 	 4.41009717114297
Replaced Outliers

114 	 4.41009717114297
Replaced Outliers

125 	 4.41009717114297
Replaced Outliers

129 	 4.41009717114297
Replaced Outliers

165 	 4.41009717114297
Replaced Outliers

185 	 4.41009717114297


In [50]:
#Rating info
dataFrame['Rating'].describe().transpose()

count    200.000000
mean       4.469231
std        0.953877
min        2.316036
25%        3.982299
50%        4.410097
75%        5.022093
max        6.922160
Name: Rating, dtype: float64

## final steps of cleaning

In [51]:
#check for unique values
dataFrame.nunique() 

Gross                167
Cost                 164
Number of Viewers    168
Year                  23
Duration             176
Outside                2
Netflix                2
Rating               152
dtype: int64

We believe that the reason we have unique values less than 200 for float column is that we filled NA values with median and mode and replaced outliers with the median, so some values should be the same. Also, by nature, some values might be equal, such as Year.

In [52]:
dataFrame.isna().sum() # WE FINALLY HAVE NO NA VALUES!!!

Gross                0
Cost                 0
Number of Viewers    0
Year                 0
Duration             0
Outside              0
Netflix              0
Rating               0
dtype: int64

### adding a Profit column

In [53]:
#Let's add a Profit column, this will be useful for our research

dataFrame['Profit (in USD)'] = dataFrame['Gross'] - dataFrame['Cost']

In [54]:
dataFrame['Profit (in USD)'].head()

0   -4.875855e+06
1   -1.563687e+06
2   -2.903600e+06
3   -1.360433e+06
4    3.158863e+05
Name: Profit (in USD), dtype: float64

In [55]:
dataFrame['Profit (in USD)'].tail()

195   -2.188706e+06
196   -4.531211e+06
197   -4.017372e+06
198   -2.986879e+06
199   -5.876864e+06
Name: Profit (in USD), dtype: float64

Hmm... Negative profits? İnteresting...

In [56]:
# display descriptives again to see the new, cleaned data
dataFrame.head()
dataFrame.tail()
dataFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gross              200 non-null    float64
 1   Cost               200 non-null    float64
 2   Number of Viewers  200 non-null    float64
 3   Year               200 non-null    float64
 4   Duration           200 non-null    float64
 5   Outside            200 non-null    object 
 6   Netflix            200 non-null    object 
 7   Rating             200 non-null    float64
 8   Profit (in USD)    200 non-null    float64
dtypes: float64(7), object(2)
memory usage: 14.2+ KB


In [57]:
dataFrame.isna().sum() # just for good measure

Gross                0
Cost                 0
Number of Viewers    0
Year                 0
Duration             0
Outside              0
Netflix              0
Rating               0
Profit (in USD)      0
dtype: int64

## Rename columns properly

It would be even nicer if column titles were more descriptive and contained units.

In [58]:
dataFrame.rename(columns={'Gross': 'Gross (in USD)',
                          'Cost': 'Cost (in USD)',
                          'Duration' : 'Duration (in minutes)',
                          'Outside' : 'Venue Outside of US?',
                          'Netflix' : 'Available on Netflix?',
                          'Rating' : 'Rating (1-10)',
                          'Number of Viewers' : 'Number of Viewers (in millions)'}, inplace=True)
dataFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 9 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Gross (in USD)                   200 non-null    float64
 1   Cost (in USD)                    200 non-null    float64
 2   Number of Viewers (in millions)  200 non-null    float64
 3   Year                             200 non-null    float64
 4   Duration (in minutes)            200 non-null    float64
 5   Venue Outside of US?             200 non-null    object 
 6   Available on Netflix?            200 non-null    object 
 7   Rating (1-10)                    200 non-null    float64
 8   Profit (in USD)                  200 non-null    float64
dtypes: float64(7), object(2)
memory usage: 14.2+ KB


In [59]:
# as the final step of the cleaning, let's print this cleaned data to an excel spreadsheet
dataFrame.to_excel('cleanDataWithProfit.xlsx')